In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pathlib

In [2]:
import os

def rename_images_in_folder(folder, prefix):
    """
    Renames all images in a specified folder with a given prefix.

    Args:
        folder (str): Path to the folder containing the images.
        prefix (str): Prefix for the new image names.
    """
    # Get a list of files in the folder
    files = os.listdir(folder)

    # Filter only files with common image extensions
    valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}
    images = [file for file in files if os.path.splitext(file)[1].lower() in valid_extensions]

    # Step 1: Add a temporary prefix to avoid conflicts
    temp_prefix = "temp_"
    for index, image in enumerate(images):
        extension = os.path.splitext(image)[1]
        current_path = os.path.join(folder, image)
        temp_name = f"{temp_prefix}{index}{extension}"
        temp_path = os.path.join(folder, temp_name)
        os.rename(current_path, temp_path)

    # Step 2: Rename files with the desired prefix
    temp_files = os.listdir(folder)  # Reload the updated file list
    temp_images = [file for file in temp_files if file.startswith(temp_prefix)]
    for index, temp_image in enumerate(temp_images, start=1):
        extension = os.path.splitext(temp_image)[1]
        temp_path = os.path.join(folder, temp_image)
        new_name = f"{prefix}{index}{extension}"
        new_path = os.path.join(folder, new_name)
        os.rename(temp_path, new_path)

    print(f"{len(images)} images have been renamed in the folder {folder}.")

# Usage examples
#rename_images_in_folder("./data/strawberry/healthy", "strawberry_healthy_")
#rename_images_in_folder("./data/strawberry/Leaf_scorch", "strawberry_leaf_scorch_")





456 images have been renamed in the folder ./data/strawberry/healthy.


In [9]:
import os
import shutil
import pathlib

# Directory paths
original_dir = pathlib.Path("./data/strawberry/alldata")
new_base_dir = pathlib.Path("./data/strawberry/strawberry_healthy_diseased_alldata_small")

# Function to create subsets
def make_subset(category, subset_name, start_index, end_index):
    """
    Creates a subset of images for a specific category.

    Args:
        category (str): Category ('strawberry_healthy' or 'strawberry_leaf_scorch').
        subset_name (str): Subset ('train', 'validation', 'test').
        start_index (int): Starting index of the images to copy.
        end_index (int): Ending index of the images to copy.
    """
    # Create the destination directory
    dir = new_base_dir / subset_name / category
    os.makedirs(dir, exist_ok=True)

    # Generate file names and copy files
    fnames = [f"{category}_{i}.JPG" for i in range(start_index, end_index + 1)]
    for fname in fnames:
        src = original_dir / fname
        dst = dir / fname
        if src.exists():  # Check if the file exists before copying
            shutil.copyfile(src, dst)

# Create subsets for each category
# Healthy
make_subset("strawberry_healthy", "train", start_index=1, end_index=182)
make_subset("strawberry_healthy", "validation", start_index=183, end_index=274)
make_subset("strawberry_healthy", "test", start_index=274, end_index=456)

# Diseased
make_subset("strawberry_leaf_scorch", "train", start_index=1, end_index=442)
make_subset("strawberry_leaf_scorch", "validation", start_index=443, end_index=663)
make_subset("strawberry_leaf_scorch", "test", start_index=664, end_index=1109)



In [10]:
from tensorflow.keras.utils import image_dataset_from_directory

data_folder = pathlib.Path('./data/strawberry/strawberry_healthy_diseased_alldata_small')

train_dataset = image_dataset_from_directory(
    data_folder / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    data_folder / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    data_folder / "test",
    image_size=(180, 180),
    batch_size=32)


Found 624 files belonging to 2 classes.
Found 313 files belonging to 2 classes.
Found 629 files belonging to 2 classes.


In [11]:
from tensorflow.keras import layers, models

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)



In [12]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="./models/convnet_from_scratch_1207.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)              

Epoch 1/30


2024-12-07 18:34:00.856147: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [624]
	 [[{{node Placeholder/_4}}]]
2024-12-07 18:34:00.857167: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [624]
	 [[{{node Placeholder/_4}}]]


20/20 [==============================] - ETA: 0s - loss: 0.7200 - accuracy: 0.6827

2024-12-07 18:34:24.861330: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [313]
	 [[{{node Placeholder/_4}}]]
2024-12-07 18:34:24.862134: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [313]
	 [[{{node Placeholder/_4}}]]


20/20 [==============================] - 28s 1s/step - loss: 0.7200 - accuracy: 0.6827 - val_loss: 0.5325 - val_accuracy: 0.7061
Epoch 2/30
20/20 [==============================] - 25s 1s/step - loss: 0.5501 - accuracy: 0.7099 - val_loss: 0.3062 - val_accuracy: 0.9553
Epoch 3/30
20/20 [==============================] - 24s 1s/step - loss: 0.4958 - accuracy: 0.8365 - val_loss: 0.1855 - val_accuracy: 0.9297
Epoch 4/30
20/20 [==============================] - 26s 1s/step - loss: 0.2147 - accuracy: 0.9119 - val_loss: 0.0626 - val_accuracy: 0.9808
Epoch 5/30
20/20 [==============================] - 25s 1s/step - loss: 0.1871 - accuracy: 0.9247 - val_loss: 0.0555 - val_accuracy: 0.9872
Epoch 6/30
20/20 [==============================] - 26s 1s/step - loss: 0.0538 - accuracy: 0.9840 - val_loss: 0.0171 - val_accuracy: 0.9904
Epoch 7/30
20/20 [==============================] - 29s 1s/step - loss: 0.0309 - accuracy: 0.9904 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 8/30
20/20 [=============

KeyboardInterrupt: 

In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
test_model = keras.models.load_model("./models/convnet_from_scratch_1207.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")